In [11]:
from pathlib import Path
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, LinearRegression

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image


In [26]:
import pandas as pd
from pathlib import Path

TRAIN_PATH = "data/processed/train_final.csv"
VAL_PATH = "data/processed/val_final.csv"
META_PATH = "data/satellite/image_metadata.csv"

train_df = pd.read_csv(TRAIN_PATH)
val_df = pd.read_csv(VAL_PATH)
meta_df = pd.read_csv(META_PATH)

print(train_df.shape, val_df.shape, meta_df.shape)


(13334, 25) (2875, 25) (16209, 3)


In [27]:
train_img_df = train_df.merge(
    meta_df[["id", "image_path"]],
    on="id",
    how="inner"
)

val_img_df = val_df.merge(
    meta_df[["id", "image_path"]],
    on="id",
    how="inner"
)

print("Train with images:", train_img_df.shape)
print("Val with images:", val_img_df.shape)


Train with images: (13494, 26)
Val with images: (2913, 26)


In [28]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
import numpy as np


In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])
class SatelliteDataset(Dataset):
    def __init__(self, df):
        self.df = df.reset_index(drop=True)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row["image_path"]).convert("RGB")
        img = image_transform(img)
        return img, row["id"]


In [30]:
resnet = models.resnet18(pretrained=True)
resnet.fc = nn.Identity()
resnet = resnet.to(device)
resnet.eval()


/opt/anaconda3/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [36]:
# Load metadata
meta_df = pd.read_csv("data/satellite/image_metadata.csv")

# Keep only valid images
meta_df = meta_df[meta_df["status"].isin(["ok", "cached"])]

# Merge image paths into train & val
train_img_df = train_df.merge(
    meta_df[["id", "image_path"]],
    on="id",
    how="inner"
)

val_img_df = val_df.merge(
    meta_df[["id", "image_path"]],
    on="id",
    how="inner"
)

print("Train with images:", train_img_df.shape)
print("Val with images:", val_img_df.shape)
train_img_df.head()


Train with images: (13494, 26)
Val with images: (2913, 26)


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,zipcode,lat,long,sqft_living15,sqft_lot15,image_path_x,image_exists,log_price,spatial_bin,image_path_y
0,9117000170,20150505T000000,268643,4,2.25,1810,9240,2.0,0,0,...,98055,47.4362,-122.187,1660,9240,/Users/khushirawat/Desktop/CDC/data/images/0.png,True,12.501142,2372_-6109,data/images/0.png
1,6700390210,20140708T000000,245000,3,2.50,1600,2788,2.0,0,0,...,98031,47.4034,-122.187,1720,3605,/Users/khushirawat/Desktop/CDC/data/images/1.png,True,12.409018,2370_-6109,data/images/1.png
2,7212660540,20150115T000000,200000,4,2.50,1720,8638,2.0,0,0,...,98003,47.2704,-122.313,1870,7455,/Users/khushirawat/Desktop/CDC/data/images/2.png,True,12.206078,2364_-6116,data/images/2.png
3,8562780200,20150427T000000,352499,2,2.25,1240,705,2.0,0,0,...,98027,47.5321,-122.073,1240,750,/Users/khushirawat/Desktop/CDC/data/images/3.png,True,12.772806,2377_-6104,data/images/3.png
4,7760400350,20141205T000000,232000,3,2.00,1280,13356,1.0,0,0,...,98042,47.3715,-122.074,1590,8071,/Users/khushirawat/Desktop/CDC/data/images/4.png,True,12.354497,2369_-6104,data/images/4.png


In [37]:
def extract_embeddings(df, split_name):
    loader = DataLoader(
        SatelliteDataset(df),
        batch_size=32,
        shuffle=False
    )

    embeddings = []
    ids = []

    with torch.no_grad():
        for imgs, batch_ids in loader:
            imgs = imgs.to(device)
            feats = resnet(imgs)
            embeddings.append(feats.cpu().numpy())
            ids.extend(batch_ids.tolist())

    emb = np.vstack(embeddings)
    emb_df = pd.DataFrame(emb)
    emb_df["id"] = ids

    out = f"data/embeddings/resnet18_{split_name}_embeddings.csv"
    emb_df.to_csv(out, index=False)
    print(f"✅ Saved {out}")

    return emb_df


In [38]:
meta_df.head()



,id,image_path,status
0,9117000170,data/images/0.png,ok
1,6700390210,data/images/1.png,ok
2,7212660540,data/images/2.png,ok
3,8562780200,data/images/3.png,ok
4,7760400350,data/images/4.png,ok


In [39]:
print(meta_df.columns.tolist())


['id', 'image_path', 'status']


In [41]:
print("train_img_df columns:")
print(train_img_df.columns.tolist())

print("\nSample rows:")
print(train_img_df.head())


train_img_df columns:
['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15', 'image_path_x', 'image_exists', 'log_price', 'spatial_bin', 'image_path_y']

Sample rows:
           id             date   price  bedrooms  bathrooms  sqft_living  \
0  9117000170  20150505T000000  268643         4       2.25         1810   
1  6700390210  20140708T000000  245000         3       2.50         1600   
2  7212660540  20150115T000000  200000         4       2.50         1720   
3  8562780200  20150427T000000  352499         2       2.25         1240   
4  7760400350  20141205T000000  232000         3       2.00         1280   

   sqft_lot  floors  waterfront  view  ...  zipcode      lat     long  \
0      9240     2.0           0     0  ...    98055  47.4362 -122.187   
1      2788     2.0           0     0  

In [43]:
# Keep the correct image path
train_img_df["image_path"] = train_img_df["image_path_y"]
val_img_df["image_path"]   = val_img_df["image_path_y"]

# Drop confusing columns
train_img_df = train_img_df.drop(columns=["image_path_x", "image_path_y"])
val_img_df   = val_img_df.drop(columns=["image_path_x", "image_path_y"])

# Final sanity check
assert "image_path" in train_img_df.columns
assert "image_path" in val_img_df.columns

print("✅ image_path fixed")
print(train_img_df[["id", "image_path"]].head())


✅ image_path fixed
           id         image_path
0  9117000170  data/images/0.png
1  6700390210  data/images/1.png
2  7212660540  data/images/2.png
3  8562780200  data/images/3.png
4  7760400350  data/images/4.png


In [44]:
train_img_emb = extract_embeddings(train_img_df, "train")
val_img_emb   = extract_embeddings(val_img_df, "val")


✅ Saved data/embeddings/resnet18_train_embeddings.csv
✅ Saved data/embeddings/resnet18_val_embeddings.csv


In [46]:
print("Train image embeddings:", train_img_emb.shape)
print("Val image embeddings:", val_img_emb.shape)

train_img_emb.head()


Train image embeddings: (13494, 513)
Val image embeddings: (2913, 513)


,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,id
0,0.194103,0.283658,0.112833,0.420651,0.285698,0.000000,0.046050,0.039251,0.000000,0.000000,...,0.010367,0.010738,0.137973,0.246683,0.000000,0.218913,0.007676,0.283981,0.001710,9117000170
1,0.603796,0.355009,1.204026,0.008824,0.241427,0.000000,0.001300,0.767738,0.415042,1.374431,...,1.545299,0.000000,0.042346,0.008975,0.051583,0.317482,3.111401,0.127758,0.499716,6700390210
2,0.054873,0.860067,0.686869,0.038836,0.366049,0.054929,0.007118,0.072951,0.005268,0.026211,...,0.074383,0.000000,0.025848,0.031698,0.373442,1.123506,1.562306,0.001235,0.137207,7212660540
3,0.061988,0.000000,0.272383,0.685905,0.105612,0.009762,0.021206,0.018204,0.040349,0.016827,...,0.045704,0.020665,0.426439,0.667691,0.163411,0.278357,0.140394,0.157529,0.161206,8562780200
4,0.137182,0.329706,1.310127,0.035737,0.173745,0.000000,0.000000,0.008150,0.183540,0.291707,...,0.712315,0.028038,0.014084,0.000000,0.099458,0.296003,3.848642,0.002644,0.374227,7760400350


In [91]:
train_fusion_df.columns = train_fusion_df.columns.map(str)
val_fusion_df.columns   = val_fusion_df.columns.map(str)
print("Train fusion shape:", train_fusion_df.shape)
print("Val fusion shape:", val_fusion_df.shape)


Train fusion shape: (14454, 537)
Val fusion shape: (3141, 537)


In [53]:
def rename_embedding_columns(df):
    new_cols = []
    for c in df.columns:
        # if column name is a number like '0', '1', ..., '511'
        if c.isdigit():
            new_cols.append(f"img_emb_{c}")
        else:
            new_cols.append(c)
    df.columns = new_cols
    return df


train_fusion_df = rename_embedding_columns(train_fusion_df)
val_fusion_df   = rename_embedding_columns(val_fusion_df)


In [54]:
img_cols = [c for c in train_fusion_df.columns if c.startswith("img_emb_")]

print("Image embedding columns:", len(img_cols))


Image embedding columns: 512


In [55]:
drop_cols = [
    "id", "price", "log_price", "date",
    "image_path", "image_exists"
]

tab_cols = [
    c for c in train_fusion_df.columns
    if c not in drop_cols and not c.startswith("img_emb_")
]

X_train_tab = train_fusion_df[tab_cols]
X_val_tab   = val_fusion_df[tab_cols]

X_train_img = train_fusion_df[img_cols]
X_val_img   = val_fusion_df[img_cols]

y_train = train_fusion_df["log_price"]
y_val   = val_fusion_df["log_price"]

print("Tabular:", X_train_tab.shape)
print("Image:", X_train_img.shape)


Tabular: (14454, 19)
Image: (14454, 512)


In [57]:
# Keep only numeric tabular columns
X_train_tab = X_train_tab.select_dtypes(include=["int64", "float64"])
X_val_tab   = X_val_tab.select_dtypes(include=["int64", "float64"])

print("Numeric tabular features:", X_train_tab.shape)


Numeric tabular features: (14454, 18)


In [58]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_tab_scaled = scaler.fit_transform(X_train_tab)
X_val_tab_scaled   = scaler.transform(X_val_tab)


In [59]:
import numpy as np

X_train_fusion = np.hstack([X_train_tab_scaled, X_train_img.values])
X_val_fusion   = np.hstack([X_val_tab_scaled, X_val_img.values])

print("Fusion train:", X_train_fusion.shape)
print("Fusion val:", X_val_fusion.shape)


Fusion train: (14454, 530)
Fusion val: (3141, 530)


In [100]:
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error

fusion_xgb = XGBRegressor(
    n_estimators=600,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="reg:squarederror",
    random_state=42
)

fusion_xgb.fit(X_train_fusion, y_train)

val_pred = fusion_xgb.predict(X_val_fusion)

r2_log = r2_score(y_val, val_pred)
rmse_log = np.sqrt(mean_squared_error(y_val, val_pred))

print("🔥 Fusion XGBoost R² (log):", r2_log)
print("🔥 Fusion RMSE (log):", rmse_log)
val_pred_clip = np.clip(val_pred, -5, 15)
y_val_clip = np.clip(y_val, -5, 15)

rmse_price = np.sqrt(
    mean_squared_error(
        np.expm1(y_val_clip),
        np.expm1(val_pred_clip)
    )
)

print("💰 RMSE (price scale):", rmse_price)


🔥 Fusion XGBoost R² (log): 0.8671113777509076
🔥 Fusion RMSE (log): 0.1941359577183385
💰 RMSE (price scale): 128593.45053063401


In [105]:
TEST_PATH = "data/processed/test2(test(1)).csv"
META_PATH = "data/satellite/image_metadata.csv"

test_df = pd.read_csv(TEST_PATH)
meta_df = pd.read_csv(META_PATH)

meta_df = meta_df[meta_df["status"].isin(["ok", "cached"])]

print("✅ Test shape:", test_df.shape)
test_img_df = test_df.merge(
    meta_df[["id", "image_path"]],
    on="id",
    how="left"
)

print("Test total rows:", test_df.shape[0])
print("Rows with images:", test_img_df["image_path"].notna().sum())

test_img_emb.columns = test_img_emb.columns.map(str)
test_img_emb = rename_embedding_columns(test_img_emb)

print("✅ Test embeddings:", test_img_emb.shape)


✅ Test shape: (5404, 20)
Test total rows: 5404
Rows with images: 71
✅ Test embeddings: (71, 513)


In [106]:
test_with_img = test_img_df[test_img_df["image_path"].notna()].copy()
test_no_img   = test_img_df[test_img_df["image_path"].isna()].copy()

print("With images:", test_with_img.shape)
print("Without images:", test_no_img.shape)


With images: (71, 21)
Without images: (5334, 21)


In [109]:
def rename_embedding_columns(df):
    df = df.copy()
    new_cols = []
    for c in df.columns:
        if isinstance(c, int):          # 👈 FIX
            new_cols.append(f"img_emb_{c}")
        else:
            new_cols.append(c)
    df.columns = new_cols
    return df
# Image embeddings
test_img_emb = extract_embeddings(test_with_img, "test")
test_img_emb = rename_embedding_columns(test_img_emb)

#X_test_img = test_img_emb[img_cols].values
X_test_fusion = np.hstack([X_test_tab_scaled, X_test_img])
test_pred_log = fusion_xgb.predict(X_test_fusion)


✅ Saved data/embeddings/resnet18_test_embeddings.csv


In [111]:
# SAFELY align tabular columns between train and test
safe_tab_cols = [c for c in tab_cols if c in test_with_img.columns]

print("Tab cols used (train ∩ test):", len(safe_tab_cols))


Tab cols used (train ∩ test): 18


In [113]:
# Rows WITH images
test_with_img = test_df.merge(
    meta_df[["id", "image_path"]],
    on="id",
    how="inner"
)

# Rows WITHOUT images
test_without_img = test_df[
    ~test_df["id"].isin(test_with_img["id"])
]

print("With images:", test_with_img.shape)
print("Without images:", test_without_img.shape)
print("Total:", len(test_with_img) + len(test_without_img))


With images: (71, 21)
Without images: (5334, 20)
Total: 5405


In [116]:
safe_tab_cols = [c for c in tab_cols if c in test_df.columns]


In [117]:
# With image
X_test_tab_img = test_with_img[safe_tab_cols].select_dtypes(
    include=["int64", "float64"]
)

# Without image
X_test_tab_noimg = test_without_img[safe_tab_cols].select_dtypes(
    include=["int64", "float64"]
)


In [118]:
X_test_tab_img_scaled = scaler.transform(X_test_tab_img)
X_test_tab_noimg_scaled = scaler.transform(X_test_tab_noimg)


In [119]:
test_img_emb = extract_embeddings(test_with_img, "test")
test_img_emb = rename_embedding_columns(test_img_emb)

X_test_img = test_img_emb[img_cols].values


✅ Saved data/embeddings/resnet18_test_embeddings.csv


In [120]:
X_test_fusion = np.hstack([X_test_tab_img_scaled, X_test_img])

test_pred_log_img = fusion_xgb.predict(X_test_fusion)
test_pred_log_img = np.clip(test_pred_log_img, -5, 15)
test_pred_price_img = np.expm1(test_pred_log_img)


In [121]:
test_pred_log_noimg = fusion_xgb.predict(
    np.hstack([
        X_test_tab_noimg_scaled,
        np.zeros((X_test_tab_noimg_scaled.shape[0], X_test_img.shape[1]))
    ])
)

test_pred_log_noimg = np.clip(test_pred_log_noimg, -5, 15)
test_pred_price_noimg = np.expm1(test_pred_log_noimg)


In [131]:
submission = pd.concat([
    pd.DataFrame({
        "id": test_with_img["id"].values,
        "predicted_price": test_pred_price_img
    }),
    pd.DataFrame({
        "id": test_without_img["id"].values,
        "predicted_price": test_pred_price_noimg
    })
]).sort_values("id")
submission = submission.drop_duplicates(subset=["id"], keep="first")
submission.to_csv("final_predictions.csv", index=False)

print("✅ final_predictions.csv saved")
print("Rows:", submission.shape)


✅ final_predictions.csv saved
Rows: (5396, 2)


In [132]:
print("test_df:", test_df.shape)
print("test_with_img:", test_with_img.shape)
print("test_without_img:", test_without_img.shape)
print("submission:", submission.shape)

missing_ids = set(test_df["id"]) - set(submission["id"])
extra_ids   = set(submission["id"]) - set(test_df["id"])

print("Missing IDs:", len(missing_ids))
print("Extra IDs:", len(extra_ids))


test_df: (5404, 20)
test_with_img: (71, 21)
test_without_img: (5334, 20)
submission: (5396, 2)
Missing IDs: 0
Extra IDs: 0


In [133]:
submission["id"].value_counts().head()


id
1000102       1
6372000280    1
6385800030    1
6384300020    1
6383000690    1
Name: count, dtype: int64

In [138]:
print("Duplicates in test_df:", test_df["id"].duplicated().sum())
print("Duplicates in test_with_img:", test_with_img["id"].duplicated().sum())
print("Duplicates in test_without_img:", test_without_img["id"].duplicated().sum())


Duplicates in test_df: 8
Duplicates in test_with_img: 1
Duplicates in test_without_img: 8


In [142]:
# ===============================
# TABULAR-ONLY TEST PREDICTIONS
# ===============================

# Use the same tabular columns as training
safe_tab_cols = [c for c in tab_cols if c in test_df.columns]

X_test_tab = test_df[safe_tab_cols].select_dtypes(
    include=["int64", "float64"]
)

# Scale using TRAINED scaler
X_test_tab_scaled = scaler.transform(X_test_tab)

print("✅ Tabular test matrix:", X_test_tab_scaled.shape)


✅ Tabular test matrix: (5404, 18)


In [144]:
from xgboost import XGBRegressor

# ===============================
# TRAIN TABULAR-ONLY XGBOOST
# ===============================

tabular_xgb = XGBRegressor(
    n_estimators=600,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="reg:squarederror",
    random_state=42
)

tabular_xgb.fit(X_train_tab_scaled, y_train)

print("✅ tabular_xgb trained")


✅ tabular_xgb trained


In [145]:
tabular_test_pred_log = tabular_xgb.predict(X_test_tab_scaled)

# Clip for safety
tabular_test_pred_log = np.clip(tabular_test_pred_log, -5, 15)

# Convert to price scale
tabular_test_pred_price = np.expm1(tabular_test_pred_log)

print("✅ Tabular predictions ready:", tabular_test_pred_price.shape)


✅ Tabular predictions ready: (5404,)


In [148]:
# Fill missing with tabular predictions
mask_missing = submission["predicted_price"].isna()

submission.loc[mask_missing, "predicted_price"] = tabular_test_pred_price[
    mask_missing.values
]

# Final checks
assert submission.shape[0] == test_df.shape[0]
assert submission["predicted_price"].isna().sum() == 0

submission.to_csv("final_predictions.csv", index=False)
print("✅ final_predictions.csv SAVED")


✅ final_predictions.csv SAVED


In [149]:
assert submission.shape[0] == test_df.shape[0]
assert submission["predicted_price"].isna().sum() == 0
print("🎯 FINAL SUBMISSION READY")


🎯 FINAL SUBMISSION READY
